In [ ]:
from pysr import PySRRegressor
import sys
import commonutils
from commonutils import ModelResults

import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
#from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from dataclasses import dataclass
#import prettyprinter as pp

from sklearn.cross_decomposition import PLSRegression
import warnings
import sys

from sklearn import preprocessing

from copy import deepcopy
import pickle

import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
import sys
sys.path.append("./CLossLr")
import customlosslr as clr

from commonutils import ModelsStore
warnings.simplefilter("ignore")

from commonfuncsforcli import *

In [ ]:
selected_basisset = ["MINIX"]
selected_functional = ["PBE"]
Y_perset = {}
X_perset = {}


print("Selected functional: ", selected_functional)
print(" Selected basis set: ", selected_basisset)


basicfeattouse = ["Potential_Energy",\
                    "Kinetic_Energy",\
                    "FINAL_SINGLE_POINT_ENERGY",\
                    "Dispersion_correction",\
                    "E(C)",\
                    "E(X)",\
                    "Two_Electron_Energy",\
                    "Nuclear_Repulsion",\
                    "One_Electron_Energy"]

labelname = "label"

howmanydifs = 3
allvalues_perset = pickle.load(open("./data/allvalues_perset.p", "rb"))
fullsetnames = pickle.load(open("./data/fullsetnames.p", "rb"))
supersetnames = pickle.load(open("./data/supersetnames.p", "rb"))
methods = pickle.load(open("./data/methods.p", "rb"))
functionals = pickle.load(open("./data/functionals.p", "rb"))
basis_sets = pickle.load(open("./data/basis_sets.p", "rb"))

featuresvalues_perset = {}
for setname in fullsetnames:
    Y_perset[setname] = []
    X_perset[setname] = {}
    featuresvalues_perset[setname] = []
    for val in allvalues_perset[setname]:
        #print("Label value: ", val[labelname])
        Y_perset[setname].append(val[labelname])
        featuresvalues_perset[setname].append({})
        for k in val:
            if k.find("energydiff") != -1:
                torm = k.replace("energydiff", "")
                for f in val[k]:
                    tocheck = f.replace(torm, "")
                    if tocheck in basicfeattouse:
                        keytouse = f.replace("-", "_")
                        keytouse = keytouse.replace("(", "")
                        keytouse = keytouse.replace(")", "")
                        featuresvalues_perset[setname][-1][keytouse] = val[k][f]

featuresnames = set()
for setname in fullsetnames:
    if len(featuresvalues_perset[setname]) != len(Y_perset[setname]):
        print("Error: len(featuresvalues_perset[", setname, "]) != len(Y_perset[", setname, "])")
        print("  ", len(featuresvalues_perset[setname]), " != ", len(Y_perset[setname]))
        
    #print("Set: ", setname, " with ", len(featuresvalues_perset[setname]), " entries")
    #print("Label len(Y_perset[setname]): ", len(Y_perset[setname]))
    for i in range(len(featuresvalues_perset[setname])):
        for f in featuresvalues_perset[setname][i]:
            for func in selected_functional:
                for basis in selected_basisset:
                    if f.startswith(func+"_"+basis):
                        if f not in featuresnames:
                            featuresnames.add(f)

featuresnames = list(featuresnames)
featuresnames.sort()
print("Using features: ")
for f in featuresnames:
    print("  ", f)

for setname in fullsetnames:
    for f in featuresnames:
        X_perset[setname][f] = []

for setname in fullsetnames:
    for i in range(len(featuresvalues_perset[setname])):
        for f in featuresnames:
            for sf in featuresnames:
                if f == sf:
                    X_perset[setname][f].append(featuresvalues_perset[setname][i][sf])
            

toreplace = {"Dispersion_correction": "DC",
                "FINAL_SINGLE_POINT_ENERGY": "FSPE",
                "Kinetic_Energy": "KE",
                "Nuclear_Repulsion": "NR",
                "One_Electron_Energy": "OEE",
                "Potential_Energy": "PE",
                "Two_Electron_Energy": "TEE"} 

Xnew = {}
for setname in fullsetnames:
    Xnew[setname] = {}
    for f in X_perset[setname]:
        newf = f
        for k in toreplace:
            newf = newf.replace(k, toreplace[k])
        if newf != f:
            Xnew[setname][newf] = deepcopy(X_perset[setname][f])
            
X_perset = {}
X_perset = deepcopy(Xnew)
featuresnames = list(X_perset[fullsetnames[0]].keys())

# chck diemnsions
for setname in fullsetnames:
    #print("Set: ", setname)
    #print("  len(Y_perset[setname]): ", len(Y_perset[setname]))
    for f in featuresnames:
        #print("   Feature: ", f, " len(X_perset[setname][f]): ", len(X_perset[setname][f]))
        if len(X_perset[setname][f]) != len(Y_perset[setname]):
            print("Set: ", setname)
            print("Error: len(X_perset[", setname, "][", f, "]) != len(Y_perset[", setname, "])")
            print("  ", len(X_perset[setname][f]), " != ", len(Y_perset[setname]))

print("Using features: ")
for f in featuresnames:
    print("  ", f)


In [ ]:
# scatterplot of features vs label
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

for setname in fullsetnames:
    for f in featuresnames:
        if "FSPE" in f:
            #plt.figure()
            #plt.scatter(X_perset[setname][f], Y_perset[setname])
            #plt.xlabel(f)
            #plt.ylabel(labelname)
            #plt.title("Set: "+setname+" Feature: "+f)
            #plt.show()
            mse = mean_squared_error(Y_perset[setname], X_perset[setname][f])
            rmse = np.sqrt(mse)
            r2 = r2_score(Y_perset[setname], X_perset[setname][f])
            mape = mean_absolute_percentage_error(Y_perset[setname], X_perset[setname][f])
            print("Set: ", setname, " Feature: ", f)
            print("  RMSE: ", rmse)
            print("    R2: ", r2)
            print("  MAPE: ", mape)
        #plt.savefig("scatter_"+setname+"_"+f+".png")
        #plt.close()